In [ ]:
import tensorflow as tf
import numpy as np
import pickle
import time
import os
import datetime
from tensorflow.contrib import learn
import data_helpers
from text_cnn import TextCNN

# Parameters

# Data loading Parameters
tf.flags.DEFINE_float('dev_sample_percetage', 0.1, 'Percetafe of training data to use for vaildation')
tf.flags.DEFINE_string('data_file', r'F:\post graduate\raw_data\clean_total.csv', 'Data Source')

# Model Hyperparams
tf.flags.DEFINE_integer('embedding', 100, 'Dimension of embedding')
tf.flags.DEFINE_string('filter_sizes', '3,4,5', 'Filter size')
tf.flags.DEFINE_integer('num_filters', 100, 'Number of filters')
tf.flags.DEFINE_float('dropout_keep_prob', 0.5, 'Dropout')
tf.flags.DEFINE_float('L2_reg_Lamda', 0.01, 'L2')

# Training Parameters
tf.flags.DEFINE_integer('batch_size', 64, 'Batch size')
tf.flags.DEFINE_integer('num_epochs', 10, 'Number of epoch')
tf.flags.DEFINE_integer('evaluate_every', 100, 'Evaluate_every')
tf.flags.DEFINE_integer('checkpoint_every', 10000, 'Saving...')
tf.flags.DEFINE_integer('num_checkpoints', 5000, 'Number of checkpoints to store')

tf.flags.DEFINE_boolean('log_device_placement', False, 'Log placement of ops on devices')
tf.flags.DEFINE_boolean('allow_soft_placement', True, 'Allow device soft device placement')
# Misc Parameters
FLAGS = tf.flags.FLAGS  # FLAGS保存命令行参数的数据
FLAGS.flag_values_dict()  # 将其解析成字典存储到FLAGS.__flags中
print('Patameters')
for k in FLAGS:
    print(k.upper(), FLAGS[k].value)


def prepocess():
    # loda data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(FLAGS.data_file)

    # bulid vocabulary
    max_document_length = max(len(x.split(' ')) for x in x_text)
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # shuffle
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # split train/test dataset
    dev_sample_index = -1 * int(FLAGS.dev_sample_percetage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print('Vocabulary Size: {:d}'.format(len(vocab_processor.vocabulary_)))
    print('Train/Dev split: {:d}/{:d}'.format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev


def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=FLAGS.allow_soft_placement,
            log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.L2_reg_Lamda
            )

        # Define Training procedure
        global_step = tf.Variable(0, name='global_step', trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        # minimize()的第一步，返回(gradient, variable)对的list
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        # minimize()的第二部分，返回一个执行梯度更新的ops
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # 初始化所有变量
        sess.run(tf.global_variables_initializer())

    def train_step(x_batch, y_batch):
        feed_dict = {
            cnn.input_x: x_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: FLAGS.dropout_keep_prob}
        _, step, summaries, loss, accuracy = sess.run(
            [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
            feed_dict)
        time_str = datetime.datetime.now().isoformat()
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
        train_summary_writer.add_summary(summaries, step)

    def dev_step(x_batch, y_batch, writer=None):
        feed_dict = {
            cnn.input_x: x_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: 1.0
        }
        step, summaries, loss, accuracy = sess.run(
            [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
            feed_dict=feed_dict
        )
        time_str = datetime.datetime.now()
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
        # if writer:
        #     writer.add_summary(summaries, step)

    # generate batches
    batches = data_helpers.batch_iter(list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

    # training loop
    for batch in batches:
        x_batch, y_batch = zip(*batch)
        train_step(x_batch, y_batch)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % FLAGS.evaluate_every == 0:
            print('\n Evaluation:')
            dev_step(x_dev, y_dev, writer=dev_summary_writer)
        # if current_step % FLAGS.checkpoint_every == 0:
        #     path = saver.save(sess, checkpoint_prefix, global_step=current_step)  # 指定保存路径
        #     print('Save model checkpoint to {} \n'.format(path))


def main(argv=None):
    x_train, y_train, vocab_processor, x_dev, y_dev = prepocess()
    train(x_train, y_train, vocab_processor, x_dev, y_dev)


if __name__ == '__main__':
    tf.app.run()
